# 3. data split by user

In [1]:
import os
from datetime import datetime as dt

import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
preprocessed_data_dir = '/dev/shm/data/3.preprocessed/'

In [3]:
all_transaction_5_df = pd.read_csv(os.path.join(preprocessed_data_dir, 'merged-data-over-5-240321.csv'))
print('all_transaction_5_df shape: ', all_transaction_5_df.shape)
all_transaction_5_df.sample(5)

all_transaction_5_df shape:  (276847, 4)


,uid,recipe_sno,rating,datetime
108938,27918181,6881990,5.0,2018-04-06 18:44
177362,78730171,6832325,5.0,2018-06-26 17:38
93952,77412926,6900988,5.0,2023-12-08 11:05
123177,58099972,4607250,5.0,2017-08-25 15:11
132639,18105397,6847221,5.0,2023-01-05 10:03


In [4]:
all_transaction_10_df = pd.read_csv(os.path.join(preprocessed_data_dir, 'merged-data-over-10-240321.csv'))
print('all_transaction_10_df shape: ', all_transaction_10_df.shape)
all_transaction_10_df.sample(5)

all_transaction_10_df shape:  (196285, 4)


,uid,recipe_sno,rating,datetime
28889,79115518,6929687,NaN,NaN
118707,67753797,6880462,5.0,2018-04-19 20:01
81459,darkngelmk,6868530,5.0,2019-11-12 06:27
16604,kpmj1,6871702,NaN,NaN
77596,77834726,6876693,5.0,2018-08-07 20:12


In [5]:
def split_by_user(df, train_ratio=.8):
	train_ratio = .8
	train_data = pd.DataFrame(columns=df.columns)
	test_data = pd.DataFrame(columns=df.columns)

	for uid, user_df in tqdm(df.groupby('uid')):

		# 시간 순으로 정렬 (na의 경우 가장 오래된 데이터 취급)
		sorted_user_df = user_df.sort_values(by='datetime', na_position='first')

		# 목표 분할 비율에 맞는 인덱스 찾기
		split_point = int(len(sorted_user_df) * train_ratio)

		# 데이터 분할
		train_data = pd.concat([train_data, sorted_user_df.iloc[:split_point]])
		test_data = pd.concat([test_data, sorted_user_df.iloc[split_point:]])
	
	return train_data, test_data

In [6]:
train_data_5, test_data_5 = split_by_user(all_transaction_5_df)
train_data_10, test_data_10 = split_by_user(all_transaction_10_df)

  0%|          | 0/20588 [00:00<?, ?it/s]/tmp/ipykernel_1366939/2540369673.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  train_data = pd.concat([train_data, sorted_user_df.iloc[:split_point]])
/tmp/ipykernel_1366939/2540369673.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  test_data = pd.concat([test_data, sorted_user_df.iloc[split_point:]])
  0%|          | 0/7992 [00:00<?, ?it/s]/tmp/ipykernel_1366939/2540369673.py:15: FutureWarning: The behavior of DataFrame concatenation with em

In [7]:
# print(train_data.uid.nunique())
# print(train_data.recipe_sno.nunique())
# print(test_data.uid.nunique())
# print(test_data.recipe_sno.nunique())

In [9]:
train_data_5.shape, test_data_5.shape

((213757, 4), (63090, 4))

In [10]:
train_data_10.shape, test_data_10.shape

((153955, 4), (42330, 4))

In [8]:
now = dt.now().strftime('%y%m%d')
train_data_5.to_csv(f'/dev/shm/data/4.split-by-user/train-data-over-5-{now}.csv', index=False)
test_data_5.to_csv(f'/dev/shm/data/4.split-by-user/test-data-over-5-{now}.csv', index=False)
train_data_10.to_csv(f'/dev/shm/data/4.split-by-user/train-data-over-10-{now}.csv', index=False)
test_data_10.to_csv(f'/dev/shm/data/4.split-by-user/test-data-over-10-{now}.csv', index=False)